# Question 2: CRUD Operations in SQL Server

Objective: \
Basic SQL Server interactions.

Task: \
Create an SQL table schema to store time-series metal prices. Include fields like Date, Metal, Price.
Demonstrate basic CRUD operations


In [9]:
import os
import sqlite3

import pandas as pd
import numpy as np

data_path = f'{os.pardir}/data/MarketData.csv'

In [10]:
# Load the data 
raw_df = pd.read_csv(data_path)
raw_df

,Start Date,01/01/2010,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,End Date,01/01/2023,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,LME COPPER 3MO ($),LME ALUMINUM 3MO ($),LME ZINC 3MO ($),LME LEAD 3MO ($),LME TIN 3MO ($),Generic 1st 'CL' Future
3,NaN,LMCADS03 Comdty,LMAHDS03 Comdty,LMZSDS03 Comdty,LMPBDS03 Comdty,LMSNDS03 Comdty,CL1 Comdty
4,NaN,Settlement Price,Settlement Price,Settlement Price,Settlement Price,Settlement Price,Settlement Price
...,...,...,...,...,...,...,...
3392,26/12/2022,8349.5,2389.5,2965,2273.5,23934,79.56
3393,27/12/2022,8349.5,2389.5,2965,2273.5,23934,79.53
3394,28/12/2022,8443,2381,3005.5,2218,24734,78.96
3395,29/12/2022,8418,2405,2984.5,2272.5,24915,78.4


In [28]:
# Remove first 6 rows (headers)
df = raw_df.iloc[6:, :].copy()
# Get columns names for metals
columns_metals = raw_df.iloc[2, 1:-1].values
columns_metals = [col.split(' ')[1] for col in columns_metals]
print(f'Metals to include in the database: {columns_metals}')

# Create a Database
conn = sqlite3.connect(f'{os.pardir}/market_data.db')
cur = conn.cursor()
# Create a table for metals
cur.execute('''
    CREATE TABLE IF NOT EXISTS MetalPrices (
        Date DATE NOT NULL,
        Metal NVARCHAR(50) NOT NULL,
        Price DECIMAL(10, 2) NOT NULL,
        PRIMARY KEY (Date, Metal)
    );
''')

# Insert data into the table, metal by metal
dates = pd.to_datetime(df.iloc[:,0], dayfirst=True).dt.strftime('%d-%m-%Y')
for m, metal in enumerate(columns_metals):
    prices = df.iloc[:, m + 1].values
    metal = [metal] * len(prices)
    data = np.column_stack((dates, metal, prices))
    cur.executemany('''
        INSERT OR REPLACE INTO MetalPrices (Date, Metal, Price)
        VALUES (?, ?, ?);
    ''', data)

cur.close()
conn.commit()
conn.close()

Metals to include in the database: ['COPPER', 'ALUMINUM', 'ZINC', 'LEAD', 'TIN']
